In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [ ]:
train=pd.read_csv('../input/msbd5001-spring-2022/train.csv')
train=train.dropna(how='any', axis=0)

In [ ]:
del train['id']
print(train.describe())

In [ ]:
train.isnull().sum(axis=0)

In [ ]:
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.countplot(x='Age', hue='label', data=train)
plt.subplot(1, 2, 2)
sns.countplot(x='Sex 0M1F', hue='label', data=train)


In [ ]:
plt.figure(figsize=(15,10))
sns.heatmap(train.corr(),annot=True)

In [ ]:
feature_list=[]
def standardize_corr_cols(data, corr_df, thresh):
    global feature_list
    for i,val in enumerate(corr_df['label'].values):
#         print(val)
        if thresh<abs(val)<1 and corr_df[corr_df.index[i]].dtype=='float64':
            feature_list.append(corr_df.index[i])
    

    for elem_feature in feature_list:
        std_scale = StandardScaler()
        data[elem_feature] = std_scale.fit_transform(data[elem_feature].values.reshape(-1,1))
        
    return feature_list
        
    
standardize_corr_cols(train, train.corr(), 0.1)  
del train['Age']    

In [ ]:
X_train=train[train.columns.tolist()[:-1]]
y_train=train['label']

In [ ]:
import sklearn.ensemble as ske
from sklearn import datasets, model_selection, tree, preprocessing, metrics, linear_model
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier,ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier

# Grid and Random Search
from sklearn.model_selection import GridSearchCV,cross_val_score,StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV

# Metrics
from sklearn.metrics import precision_recall_fscore_support, roc_curve, auc
kfold=StratifiedKFold(n_splits=10)


classifiers=[]
classifiers.append(SVC())
classifiers.append(DecisionTreeClassifier())
classifiers.append(RandomForestClassifier())
classifiers.append(ExtraTreesClassifier())
classifiers.append(GradientBoostingClassifier())
classifiers.append(KNeighborsClassifier())
classifiers.append(LogisticRegression())
classifiers.append(LGBMClassifier())
# compute the mean and std score of each model by cv search
cv_results=[]
for classifier in classifiers:
    cv_results.append(cross_val_score(classifier,X_train,y_train,
                                      scoring='accuracy',cv=kfold,n_jobs=-1))

In [ ]:
classifiers

In [ ]:
# sum up all the result
cv_means=[]
cv_std=[]
for cv_result in cv_results:
    cv_means.append(cv_result.mean())
    cv_std.append(cv_result.std())
    
cvDf=pd.DataFrame({'cv_mean':cv_means,
                     'cv_std':cv_std,
                     'algorithm':['SVC','DecisionTreeCla','RandomForestCla','ExtraTreesCla',
                                  'GradientBoostingCla','KNN','LR','LGBMClassifier']})

cvDf = cvDf.sort_values('cv_mean',ascending=False)
cvDf

In [ ]:
classifiers

In [ ]:
# deal with test data
test=pd.read_csv('../input/msbd5001-spring-2022/test.csv')
# train=train.dropna(how='any', axis=0)
del test['Age']
del test['id']

def standardize_cols(data, feature_list):
    for elem_feature in feature_list:
        std_scale = StandardScaler()
        data[elem_feature] = std_scale.fit_transform(data[elem_feature].values.reshape(-1,1))

standardize_cols(test, feature_list)
test

In [ ]:
# train each classifier one by one
result_dict={}
for i, classifier in enumerate(classifiers):
    classifier.fit(X_train, y_train)
    res=classifier.predict(test)
    result_dict.update({i:res})
    
result_dict

In [ ]:
# merge the result of every model
all_res_df=pd.DataFrame(result_dict)
tmp_dict=dict(all_res_df.T)

res_list=[]
for key in tmp_dict.keys():
    if len(tmp_dict[key].value_counts())==1:
        res_list.append(tmp_dict[key][0])
    elif tmp_dict[key].value_counts()[0]>=4:
        res_list.append(0)
    else:
        res_list.append(1)


In [ ]:
submissions=pd.DataFrame({'id':np.array([i for i in range(len(test))]), 'label':np.array(res_list)})
submissions.to_csv('submissions.csv', index=None)